In [1]:
import pandas as pd
import numpy as np

In [4]:
complete_df = pd.read_csv('cleaned_df.csv')

TypeError: read_csv() got an unexpected keyword argument 'ignore_index'

In [9]:
complete_df = complete_df.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis = 1)

In [10]:
complete_df.columns

Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion', 'data_id',
       'dcoilwtico', 'city', 'state', 'type', 'cluster', 'transactions',
       'month', 'day', 'year', 'was_holiday'],
      dtype='object')

In [17]:
columns = [ 'sales', 'onpromotion',
       'dcoilwtico', 'city', 'state', 'type', 'cluster', 'transactions',
       'month', 'day', 'year', 'was_holiday']
pre_model_df = complete_df[columns]


In [20]:
pre_model_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3029400 entries, 0 to 3029399
Data columns (total 13 columns):
 #   Column        Dtype  
---  ------        -----  
 0   sales         float64
 1   onpromotion   int64  
 2   data_id       object 
 3   dcoilwtico    float64
 4   city          object 
 5   state         object 
 6   type          object 
 7   cluster       int64  
 8   transactions  float64
 9   month         int64  
 10  day           int64  
 11  year          int64  
 12  was_holiday   bool   
dtypes: bool(1), float64(3), int64(5), object(4)
memory usage: 280.2+ MB


In [19]:
target = pre_model_df['sales']
features = pre_model_df.drop('sales', axis = 1)

In [15]:
# one hot encode and scale data

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# select columns to encode or scale
cat_columns = features.select_dtypes(include = ['object'])
num_columns = features.select_dtypes(include = ['float64','int64'])

# instanciate transformers
ohe_transformer = OneHotEncoder(drop = 'first')
num_transformer = StandardScaler()

# combine transformers
preprocessor = ColumnTransformer(
    transformers = [
        ('cat', ohe_transformer, cat_columns),
        ('num', num_transformer, num_columns)
    ])

# create pipeline
pipe = Pipeline(steps = [('preprocessor', preprocessor)])

features_processed = pipe.fit_transform(features)

# convert back to df
column_names = (
    categorical_transformer.get_feature_names_out(cat_columns) + num_columns
    passthrough_cols
)

features_processed_df = pd.DataFrame(features_processed, columns = columns_names)